In [ ]:
#Importing the Excel file into Python

import pandas as pd
import numpy as np
import statsmodels.api as sm

# Replace 'your_file.csv' with the path to your actual CSV file
data = pd.read_excel("G:/My Drive/Ireland - Jasleen/Trinity/Coursework/Dissertation/Drafts/Analysis and Findings/Country Wise Analysis/NIFTY 50 CleanData.xlsx")

# Check for missing values and remove rows with missing returns
data = data.dropna(subset=['Return'])

# Selecting specific columns from the dataset
data = data[['Date', 'PX_LAST', 'Return','Day of the week']]

# Display the first few rows of the dataframe
print(data.head(40))

In [ ]:
# Specify the start and end dates of the desired range
start_date = '2016-04-25'
end_date = '2024-05-10'

# Filter the dataframe based on the date range
data = data.loc[(data['Date'] >= start_date) & (data['Date'] <= end_date)]

# Print the filtered dataframe
# print(data.head(40))

In [ ]:
import pandas as pd

# Load the data from the provided Excel file
# file_path = '/mnt/data/NIFTY_50_Data_with_Week_Column.xlsx'
# data = pd.read_excel(file_path)

# Ensure the 'Date' column is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Identify the first trading day of the month
data['Month'] = data['Date'].dt.to_period('M')
first_trading_day_of_month = data.groupby('Month')['Date'].transform('min')
data['First_Trading_Day'] = first_trading_day_of_month

# Define the function to calculate the week of the month
def get_week_of_month(row):
    date = row['Date']
    # first_day_of_month = date.replace(day=1)
    first_trading_day = row['First_Trading_Day']

    # Check if the first trading day of the month is a Monday
    if first_trading_day.weekday() == 0:  # Monday is represented by 0
        # Apply specific formula for all dates in months that begin with Monday
        # first_monday_of_month = first_trading_day
        return ((date - first_trading_day).days // 7) + 1
    else:
        # Find the first Monday of the month
        first_monday_of_month = first_trading_day + pd.offsets.Week(weekday=0)
        if date < first_monday_of_month:
            return 1
        else:
            return ((date - first_monday_of_month).days // 7) + 2

# Apply the function to calculate the week of the month for each date
data['Week_of_Month'] = data.apply(get_week_of_month, axis=1)

# Save the modified DataFrame to a new Excel file
# output_file_path = '/mnt/data/NIFTY_50_Data_with_Week_of_Month_Corrected.xlsx'
# data.to_excel("G:/My Drive/Ireland - Jasleen/Trinity/Coursework/Dissertation/Drafts/Analysis and Findings/Model 2/Final Data File.xlsx", index=False)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Corrected Trading Data with Week of Month", dataframe=data)

# Displaying the first few rows to verify
data.head(60)


In [ ]:
# Filter the data to include only Mondays
monday_data = data[data['Day of the week'] == "Monday"].copy()

# Specify the start and end dates of the desired range
start_date = '2016-05-02'
end_date = '2024-05-10'

# Filter the dataframe based on the date range
monday_data = monday_data.loc[(monday_data['Date'] >= start_date) & (monday_data['Date'] <= end_date)]

print(monday_data.head(30))

In [ ]:
# Create dummy variables for each week of the month
week_dummies = pd.get_dummies(monday_data['Week_of_Month'], prefix='Week').astype(int)

# Assuming `week_dummies` is the DataFrame containing the dummy variables
# week_dummies = week_dummies.astype(int)

# Print the updated DataFrame
# print(week_dummies.head(10))

# Concatenate the dummy variables with the original dataframe
monday_data = pd.concat([monday_data, week_dummies], axis=1)

# Print the updated dataframe
print(monday_data.head(60))

In [ ]:
# Define the response variable (returns)
Y_1 = monday_data['Return']  # Assuming 'Return' is the column name for returns

# Define the explanatory variables (dummy variables)
X_1 = monday_data[['Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5']]

print(monday_data.dtypes)

In [ ]:
#ANOVA model

from statsmodels.formula.api import ols
import statsmodels.api as sm

# Create the ANOVA model
ANOVAmodel = ols('Y_1~X_1', data=monday_data).fit()

# Perform ANOVA
anova_table = sm.stats.anova_lm(ANOVAmodel)

# Print the ANOVA table
print(anova_table)

In [ ]:
# Perform OLS regression
model1 = sm.OLS(Y_1, X_1).fit()

# Print the summary of the regression
print(model1.summary())

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey, het_arch
# Add a constant column to the exog variable
exog_with_constant = sm.add_constant(model1.model.exog)

# Perform the Breusch-Pagan test with the updated exog variable
bp_test = het_breuschpagan(model1.resid, exog_with_constant)
bp_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Pagan test:", dict(zip(bp_labels, bp_test)))

# Breusch-Godfrey test for autocorrelation
bg_test = acorr_breusch_godfrey(model1, nlags=1)
bg_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Godfrey test:", dict(zip(bg_labels, bg_test)))



In [ ]:
# Newey-West standard errors - Corrects for autocorrelation
nw_summary = model1.get_robustcov_results(cov_type='HAC', maxlags=1)
print(nw_summary.summary())

#Addressing heteroscedasticity
robust_summary = model1.get_robustcov_results(cov_type='HC1')
print(robust_summary.summary())

In [ ]:
# DURING COVID DATA

# Defining data range
# Specify the start and end dates of the desired range
start_date = '2020-01-01'
end_date = '2021-12-31'

# Filter the dataframe based on the date range
duringcovid_data = monday_data.loc[(monday_data['Date'] >= start_date) & (monday_data['Date'] <= end_date)]

# Print the selected columns
print(duringcovid_data)

# Define the response variable (returns)
Y_2 = duringcovid_data['Return']  # Assuming 'Return' is the column name for returns

# Define the explanatory variables (dummy variables)
X_2 = duringcovid_data[['Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5']]

print(duringcovid_data.dtypes)

# Add a constant term for the intercept
# X_2 = sm.add_constant(X_2)


#ANOVA model

from statsmodels.formula.api import ols
import statsmodels.api as sm

# Create the ANOVA model
ANOVAmodel = ols('Y_2~X_2', data=duringcovid_data).fit()

# Perform ANOVA
anova_table = sm.stats.anova_lm(ANOVAmodel)

# Print the ANOVA table
print(anova_table)

# Perform OLS regression
model2 = sm.OLS(Y_2, X_2).fit()

# Print the summary of the regression
print(model2.summary())

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey, het_arch
# Add a constant column to the exog variable
exog_with_constant = sm.add_constant(model2.model.exog)

# Perform the Breusch-Pagan test with the updated exog variable
bp_test = het_breuschpagan(model2.resid, exog_with_constant)
bp_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Pagan test:", dict(zip(bp_labels, bp_test)))

# Breusch-Godfrey test for autocorrelation
bg_test = acorr_breusch_godfrey(model2, nlags=1)
bg_labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print("Breusch-Godfrey test:", dict(zip(bg_labels, bg_test)))



In [ ]:
# Newey-West standard errors - Corrects for autocorrelation
nw_summary = model2.get_robustcov_results(cov_type='HAC', maxlags=1)
print(nw_summary.summary())

#Addressing heteroscedasticity
robust_summary = model2.get_robustcov_results(cov_type='HC1')
print(robust_summary.summary())